# Template für K-means Clustering | Single-Node/Single-GPU

***

In diesem Abschnitt wird erkläre, wie man den KMeans Algorithmus auf Worker skaliert und zum Laufen bringt. Dazu nutzen wir Dask und Rapids. Dask bietet uns ein Dashboard wo wir Einzelheiten verfolgen können. Dask-Cuda lässt sich gut mit PyTorch und anderen Frameworks kombinieren, ohne die Clustereinstellungen groß zu ändern.

In dem Nootebook für das Setup wird erklärt, wie man selber sowas auf seinem eigenen PC/Workstation/Cluster installiert. Es gibt auch andere Möglichkeiten das zu installieren. Hier nutzen wir ein Cluster Setup mit Nodes, installiert mittels Docker.   

<u>Hinweis:</u><br>
Wenn es mal hängt, muss ggf. der Cluster, Worker oder das Notebook neu gestartet werden. Sowas passiert, wenn z.B. eine Exception vorkommt.   

<br>

Dask: https://www.dask.org | https://docs.dask.org/en/stable/ <br>
Rapids: https://rapids.ai <br> 

# 1. Rapids und Dask

<img src="./pictures/raspids_oss.PNG"  width="825px;" hight="725px;">
(Bild: (https://on-demand.gputechconf.com/gtcdc/2018/pdf/dc8256-rapids-the-platform-inside-and-out.pdf))

Rapids bietet uns eine Sammlung von Libraries, die uns bei vielen Dingen unterstützen kann. Eine dieser Libraries, die wir nutzen werden, nennt sich cuDF. cuDF sind Cuda Dataframes, die erstmal nur auf einer GPU laufen. Die Idee dahinter ist, dass wir <u>nah zu dasselbe machen</u> wie bei Pandas (Panda DataFrame), aber nur auf einer GPU. Dadurch können viele Berechnungen viel schneller durchgeführt werden. Beispielsweise das Vorverarbeiten von Daten vor dem Trainieren, oder anderweitige Zwecke. Die API ist nah zu identisch wie bei Pandas. 

Rapids funktioniert <u>nur</u> mit <u>Nvidia Grafikkarten</u>.

Rapids ist OpenSource, die API Beschreibungen der Libraries verlinken den dahinterliegenden Code. Der Code enthählt selber nochmal Kommentare und erklärungen.

Rapids entwickelt sich und es kommen Updates sowie Erweiterungen. Algorithmen, die noch nicht (Single-/Multi-Node/ GPU) verfügbar sind, können noch in den neueren Versionen auftauchen. Das gilt besonders für Multi-Node/Multi-GPU Implementierungen der Algorithmen. Es kann sein das manche Multi-Node/Multi-GPU Algorithmen sich am Ende nicht zusammenfassen lassen können, oder das Speichern/Laden sowie die Verwendung dieser auf GPU/CPU eingeschränkt sind. Andere Multi-Node/Multi-GPU Algorithmen können so wie in diesem Template hier verwendet werden. Einge andere können sich in der Anwendung unterscheiden. 


Floating-Point Berechnungen mit cuDF werden auf der GPU parallel ausgeführt, also ist die Reihenfolge nicht deterministisch. Die Ergebnisse können von Pandas abweichen.
a + b ist nicht gleich b + a. Das kann z.B. bei data.sum() vorkommen.
cuDF unterstützt auch verschiedene Datentypen wie int32, int64, float32, float64. Die ganze Liste ist hier zu finden.

Es gibt noch Dask-cuDF, diese nutzt man unter anderem, um große Daten zu Partitionieren. Die API ist Dask-ähnlich. 

Eine weitere Library von Rapids ist cuML, welche auch cuDF nutzt. Diese enthält verschiedene supervised und unsupervsed Algorithmen. Der Fokus hier liegt dabei, dass die Algorithmen möglichst auf einer GPU laufen. Ein Teil dieser Algorithmen kann mittels eines Clusters (cuML nutzt dafür ein Dask Cluster) auf verschiedene Nodes und GPUs skalieren. 

Die API von cuML ist fast genauso aufgebaut wie bei 
Sklearn. <br> Die Release Notes von cuML sind hier zu finden (https://github.com/rapidsai/cuml/releases).
  

Was wir neben Rapids und Dask noch nutzen können ist CuPy [5]. CuPy ist wie Numpy, nur läuft es unter auf GPU. CuPy wird hier nicht detailliert erklärt.

<br> 

cuDF Cuda DataFrames Dok.:  https://docs.rapids.ai/api/cudf/stable/ <br>
cuDF API: https://docs.rapids.ai/api/cudf/stable/api_docs/api/cudf.dataframe/ <br>
cuDF Floating-Point Berechnung: https://docs.rapids.ai/api/cudf/stable/user_guide/pandas-comparison/#floating-point-computation        <br>
cuDF Datentypen: https://docs.rapids.ai/api/cudf/stable/user_guide/data-types/ | https://docs.rapids.ai/api/cudf/stable/user_guide/io/ <br>

cuML GitHub: https://github.com/rapidsai/cuml  <br>
cuML Dok.: https://docs.rapids.ai/api/cuml/stable/ <br>

Dask-cuDF: https://docs.rapids.ai/api/dask-cudf/stable/

CuPy: https://cupy.dev



Was wir nutzen:<br>
cuDF: DataFrames deren API nahezu identisch mit Pandas ist. Also quasi Pandas auf GPU.<br>
dask_cudf: Dask bietet uns DataFrames, die verteilt sind. Damit kann man auch Daten partitionieren [11] <br>
cupy: wie Numpy und Scipy aber auf GPU.

<img src="./pictures/df.PNG"  width="725px;" hight="725px;">

Wir teilen das Trainieren in 4 Bereiche auf. Hier nutzen wir SNSG.

Das Bild zeigt in welchen Bereichen man welche DataFrames nutzen kann. Pandas liegt dabei nicht im Fokus.

<img src="./pictures/zuordnung.PNG"  width="1050px;" hight="1050px;">

Da wir mit Dask-cuDF und Dask große Daten lesen und partitionieren können, ist auch der Einsatz im SNSG Bereich möglich. cuDF unterstützt das lesen mehrere Dateien (bis jetzt)  nicht, so das man Dask oder Dask-cuDF nutzen muss, wenn man globstings nutzt, um alle CVS Dateien in einem Verzeichnis zu lesen.

Es gibt auch die Möglichkeit cuDF als Backend für Dask zu nutzen.

<br>

cuDF Backend für Dask: https://docs.rapids.ai/api/dask-cudf/stable/#dataframe-creation-from-on-disk-formats

<b>[Bild Konvertierungen]</b>


<u>Hinweis:</u><br>
Variablen die GPU Speicher nutzen und nicht mehr gebraucht werden, sollten mittels `del` gelöscht werden. Wenn nach dem Löschen immer noch Speicher belegt ist, muss der Kernel des Notebooks neugestartet werden. 

In [2]:
# Imports 
import cupy as   cp
import cudf
import cuml
from cuml.dask.common import utils as dask_utils
import dask_cudf
import dask

import numpy as np
from numpy import genfromtxt

import pandas as pd
import os
import time

Da das Interface wie Pandas aufgebaut ist, wird cuDF im detail nicht erklärt. Bei interesse kann man sich die API durchlesen.

In [3]:
# cuDF und Panda series 
s_cuda = cudf.Series([1, 2, cudf.NA])
s_panda = pd.Series([1, 2, pd.NA])

print(f"Pandas:\n{s_cuda}\n")
print(f"Rapids:\n{s_panda}")

Pandas:
0       1
1       2
2    <NA>
dtype: int64

Rapids:
0       1
1       2
2    <NA>
dtype: object


## 1.1 Start Dask-Cluster (Mit Docker) 

Pro GPU wird genau ein Prozess für einen Worker gestartet. Die Worker-ID wird hochgezählt. <br>

<u>Hinweise:</u><br>
Die MNMG Versionen der Algorithmen können auch auf SNSG laufen.

Wenn wir nur auf einen Knoten sind, können wir normal das Cluster-Setup nutzen oder einen lokalen Cluster erstellen. Hier nutzen wir den normalen Cluster mit einem GPU Worker.

<img src="./pictures/rapids_cluster.PNG">

In [4]:
import rapids_tools as rapids_tools   

Rapids-tools 1.0
Umgebungsvariablen werden gesetzt
NCCL_SOCKET_NTHREADS: 4
NCCL_NSOCKS_PERTHREAD: 2


In [5]:
# Erstelle Client 
client = rapids_tools.create_dask_client()  # Gebe IP an. Standard:  127.0.0.1:8786 (Scheduler ist da wo auch Jupyter läuft) 
client                                                           

Client IP: 127.0.0.1:8786


<Client: 'tcp://149.201.182.203:8786' processes=1 threads=1, memory=62.53 GiB>

Wenn wir eigene Python Module erstellen, können wir diese auch mit dem Cluster teilen.

In [6]:
# Lade Datei mit Dask hoch. Gibt Status aus... 
client.upload_file('rapids_tools.py')  # Nur als Beispiel

{'tcp://149.201.182.203:38309': {'status': 'OK'}}

In [7]:
# Zeige Schlüssel. Die Ports ändern sich. 
client.has_what().keys()

dict_keys(['tcp://149.201.182.203:38309'])

In [8]:
# Starte alle Worker neu
client.restart_workers(client.has_what().keys())
client.has_what().keys()

dict_keys(['tcp://149.201.182.203:42963'])

In [9]:
# Startet client neu.
#client.restart()

# 2. Daten

## 2.1 Daten generieren

Als Anfang erstellen wir Daten. Daten die auf dem Host sind, müssen erst verteilt werden.

cuML bietet uns Möglichkeiten Datensets zu erstellen.

In [11]:
# Imports 
from cuml import datasets            # Um Beispiele auf dem Host zu generieren
from cuml.model_selection import train_test_split

Ein Weg an Daten zu kommen, ist Datensets zu nutzen, die schon mit den Frameworks kommen. 

Das können wir nutzen, um den Aufbau des Codes zu testen und zu überprüfen, ob die Installation von Rapids und die Dask-Worker funktionieren.

Es gibt einige Parameter, die wir eingeben können, um Daten so zu generieren wie wir es wollen. Um es einfach zu halten, nutzen wir nur 2 Parameter.

<br>

Generiere Single-GPU Daten: https://docs.rapids.ai/api/cuml/stable/api/#dataset-generation-single-gpu

In [84]:
# Trainings- und Testgröße
train_size = 20000 # 20000
test_size = 100
n_samples = train_size + test_size

# Erstelle X und y Daten für ein Cluster Problem 
X, y = datasets.make_blobs(n_samples=n_samples, n_features=2)
X, X_test, y, y_test = train_test_split( X, y, train_size=0.70, random_state=5)

Jetzt können wir uns anschauen was gemacht wurde, so könnten unsere Trainingsdaten auch aussehen.

In [13]:
print(f"X: {type(X)} y: {type(y)} \n X[0]: {X[0]}  \n y[0]: {y[0]} \n Länge X, y: {len(X)}, {len(y)}")

X: <class 'cupy.ndarray'> y: <class 'cupy.ndarray'> 
 X[0]: [ 1.8046851 -6.573599 ]  
 y[0]: 1.0 
 Länge X, y: 14070, 14070


## 2.3 Daten sind schon Lokal verfügbar

Mit cuDF können wir Daten direkt in die GPU laden und Daten schreiben. Es gibt viele Parameter, die angegeben werden können. 

Wenn die Daten nicht in den Speicher passen, können diese partitioniert werden, indem wir die Daten mit Dask-cuDF laden.

<img src="./pictures/rapids_chunk.PNG">

Wir können Daten mit Dask laden und daraus ein Dask-cuDF erstellen.

Intern nutzt jeder Worker cuDF um die Daten zu lesen. Es können verschiedene Quellen angegeben werden wie HDFs, S3, http, ... (https://docs.dask.org/en/stable/how-to/connect-to-remote-data.html)

<br>

cuDF Input/Output: https://docs.rapids.ai/api/cudf/legacy/api_docs/io/#input-output <br>
Dask-cuDF Creating and storing DataFrames: https://docs.rapids.ai/api/dask-cudf/stable/api/#creating-and-storing-dataframes

In [6]:
df = cudf.DataFrame(
    {
        "a": list(range(20)),
        "b": list(reversed(range(20))),
        "c": list(range(20)),
    })

In dem Fall, dass die Daten sehr groß sind oder wir viele Daten lesen wollen, können wir alternativ zu cuDF auch Dask-cuDF nutzen. Je nach Version unterstützt cuDF noch nicht das Lesen mehrere Dateien.

Die Anwendung von Dask-cuDF ist einfach. Wer Dask schon kennt, die API von Dask-cuDF ist nahzu identisch mit Dask.

Beim lesen können wir eine Blocksize angeben. Dann wird eine Anzahl an Partitionen erstellt. 

Für ein kurzes Beispiel, schreiben wir eine Datei und lesen diese mit Dask-cuDF.

In [86]:
# save data
np.savetxt("csv_file.csv", cp.asnumpy(X), delimiter=",", header="A,B")

In [87]:
# "*.csv" um alle CSV Dateien zu lesen
dcf = dask_cudf.read_csv("csv_file.csv", blocksize="100KB")  
dcf

,# A,B
npartitions=8,,
,float64,float64
,...,...
...,...,...
,...,...
,...,...


In [88]:
type(dask_cdf)

dask_cudf.core.DataFrame

Für unseren Versuch haben wir die Trainingsdaten X. Beim lesen mit dask_cudf versucht der Worker dann den gegebenen Pfad zu lesen. Wir arbeiten ggf. in einem anderen Verzeichnis.
Damit der Worker die Datei sieht, muss der Worker diese schreiben.

In [89]:
@dask.delayed
def write_data(X):
    import cupy as cp
    np.savetxt("csv_file.csv", cp.asnumpy(X), delimiter=",", header="A,B")
fut = write_data(X)
fut.compute()

Jetzt hat dieser Worker die Datei und wir können die Datei lesen.

In [90]:
dcf = dask_cudf.read_csv("csv_file.csv", blocksize="100KB")  

dcf = dask_df.persist()
dcf

,# A,B
npartitions=8,,
,float64,float64
,...,...
...,...,...
,...,...
,...,...


In [91]:
type(dask_df)

dask_cudf.core.DataFrame

Mit Persist sagen wir, das die Daten aktiv im Speicher gehalten werden.

In [92]:
# Damit sehen wir die Patitionen
client.has_what()

{'tcp://149.201.182.203:42963': ("('read-csv-ed8861c034a65772f82ec2304d734821', 0)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 1)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 2)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 3)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 4)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 5)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 6)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 7)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 8)",
  "('read-csv-ed8861c034a65772f82ec2304d734821', 9)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 0)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 1)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 10)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 11)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 12)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 13)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 14)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 15)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 16)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 17)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 2)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 3)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 4)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 5)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 6)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 7)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 8)",
  "('read-csv-1a1e3c72740a68a8e10cb0a911b14d54', 9)")}

### Trainieren

Mit den partitionierten Daten können wir jetzt das Model trainieren.

In [93]:
%time single_model.fit(dcf)  

CPU times: user 17 ms, sys: 4.18 ms, total: 21.2 ms
Wall time: 37.6 ms


KMeans()

Eine Alternative wäre, die Partitionen stückweise zu nutzen. 

In [94]:
%%time
# Alternativ: 

partitions = 6  # Anzahl partitionen
single_model = single_Kmeans(init="k-means||", random_state=100)

for i in range(partitions):
    # Gehe Partitionen durch.
    # - So können pro Iteration Daten ausgegeben werden. 
    single_model.fit(dcf.get_partition(i))
    

CPU times: user 75.3 ms, sys: 99 µs, total: 75.4 ms
Wall time: 142 ms


# 3. Training

Wie bereits erwähnt, ist die API fast genau so wie bei Sklearn. Wenn wir jetzt Daten haben und diese noch verarbeiten wollen, muss darauf geachtet werden, dass die Datentypen am Ende richtig sind.


Für das Model gibt es einige Parameter, die wir einstellen können, dieser Auszug zeigt alle möglichen Parameter <br>
<i>handle=None, n_clusters=8, max_iter=300, tol=0.0001, verbose=False, random_state=1, init='scalable-k-means++', n_init=1, oversampling_factor=2.0, max_samples_per_batch=32768, output_type=None</i>
* n_clusters: The number of centroids or clusters you want.
* max_iter: The more iterations of EM, the more accurate, but slower.
* tol: Stopping criterion when centroid means do not change much.

<br>

Sklearn Kmeans: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html <br>
cuML SNSG Kmeans: https://docs.rapids.ai/api/cuml/stable/api/#k-means-clustering

In [95]:
# Single Node GPU K-Means
from cuml import KMeans as single_Kmeans

Jetzt kann man wie üblich vorgehen. Das Model Trainieren, Speichern, Laden und Evaluieren.

Wir können das Dashboard von Dask öffnen und sehen unter GPU die Auslastung (wenn es nicht zu schnell fertig ist).

In [96]:
# Erstelle Model mit Parameter 
single_model = single_Kmeans(init="k-means||", random_state=100)
                     
# Training  
%time single_model.fit(X)

CPU times: user 12.5 ms, sys: 12.1 ms, total: 24.5 ms
Wall time: 21.7 ms


KMeans()

Nach dem Trainieren können wir Predictions machen.

In [97]:
to_predict = X_test
print(f"Länge: {len(to_predict)} \nTyp: {type(to_predict)}\n {to_predict}")

Länge: 6030 
Typ: <class 'cupy.ndarray'>
 [[  3.1369233  -9.806425 ]
 [  1.5548234   4.953351 ]
 [  1.3772724   3.4159353]
 ...
 [  3.392383    3.2132409]
 [  0.8432759 -10.015812 ]
 [  2.218586  -10.843707 ]]


In [98]:
single_kmeans_pred = single_model.predict(to_predict)
# Als Output bekommen wir ein cupy Array
type(single_kmeans_pred)

cupy.ndarray

# 3. Speichern, Laden, Testen

Das Speichern und Laden des Modelsgeht einfach mit nur wenigen Schritten. Das Evaluieren ist ähnlich wie bei Sklearn.

In [99]:
import pickle
# Speichern
pickle.dump(single_model, open("template_data_Kmeans/cuml_SingleNode_kmeans_model.pkl", "wb"))

# Laden:
loaded_model = pickle.load(open("template_data_Kmeans/cuml_SingleNode_kmeans_model.pkl", "rb"))

In [100]:
from sklearn.manifold import trustworthiness   # CPU Berechnung 
# Oder auch:
from cuml.metrics.trustworthiness import trustworthiness as cuml_trustworthiness   # GPU Berechnung, Viel schneller

Wir können z.B. die trustworthiness Anwenden.

Wenn wir Sklearn für die trustworthiness nutzen wollen, muss der Typ des Inputs passen. Unsere Daten sind in dem Format cupy.ndarray.

Um Numpy zu nutzen, müssen wir nur  `.get()` hinter unsere Variablen schreiben, wie: `y.get()`. Oder wir wandeln die Daten vorher in ein Numpy um.

In [48]:
print(f"Type X: {type(X)}\nType X.get(): {type(X.get())}")

Type X: <class 'cupy.ndarray'>
Type X.get(): <class 'numpy.ndarray'>


In [49]:
# cuML trustworthiness, hier kein .get() nötig
X_embedded = loaded_model.transform(X)  
%time score = cuml_trustworthiness( X, X_embedded )
print(f"Score: {score}")

CPU times: user 129 ms, sys: 0 ns, total: 129 ms
Wall time: 128 ms
Score: 0.9999686574341249


# 4. cuML Kmeans => Sklearn Kmeans Model

In [101]:
from sklearn.cluster import KMeans as sk_Kmeans

Das Model bietet keine normale Überführung in ein CPU-Model. Es ist jedoch möglich, einige Werte dem Sklearn Model zu übergeben.

Ist das Ziel das Model später auf einem System zu nutzen, das keine CPU hat, kann das trainierte Model in ein Sklearn Model "konvertiert" werden.

In [102]:
# Erstelle Basismodel
sk_model_new =  sk_Kmeans()

In [103]:
sk_model_new  = rapids_tools.snsg_kmeans__singleModel_to_sklearnModel(loaded_model, sk_model_new)

Überführe cuML Model zu Sklearn Model.


Danach sollte eine Prediction möglich sein.

In [104]:
sk_model_new.predict(np.asarray( [ [1.5, 2.4] ], dtype=np.float32 ))

array([5], dtype=int32)

Dann können wir das Model speichern und wieder Laden.

In [105]:
# Speichern 
pickle.dump(sk_model_new, open("template_data_Kmeans_Multi/sklearn_kmeans_model.pkl", "wb"))
# Laden
loaded_sklearn_model = pickle.load(open("template_data_Kmeans_Multi/sklearn_kmeans_model.pkl", "rb"))

In [106]:
loaded_sklearn_model.predict(np.asarray( [ [1.5, 2.4] ], dtype=np.float32 ))

array([5], dtype=int32)

In [107]:
# cuML trustworthiness
X_embedded = loaded_sklearn_model.transform(X_test.get())  
%time score = cuml_trustworthiness(X_test, X_embedded )
print(f"Score: {score}")

CPU times: user 10.2 ms, sys: 69 µs, total: 10.2 ms
Wall time: 10.1 ms
Score: 0.9998888046207576


Danach können wir die Predictions prüfen, ob beide Modelle die gleichen Ausgeben haben. Der Input muss vom Typ Numpy sein.

In [108]:
# Predictions des Sklearn Models
single_kmeans_sklearn_pred = loaded_sklearn_model.predict(X_test.get())

In dem Fall das es eine Ungleiheit gibt, kommt eine Meldung.

In [109]:
rapids_tools.arrays_equal( cp.asnumpy( single_kmeans_pred ), single_kmeans_sklearn_pred )

Gebe 10 Zeilen aus, die ungleich sind


In [59]:
# Shutdown Cluster
client.shutdown()